## Simple Traffic Experiments

We vary different aspects of the randomized simple traffic distribution to see the effect they have on the difference between the ST solver than DDPG and the combination of the two. We have the "default" simple traffic, plus four variants. Their descriptions are:

1. Default simple traffic: Cars 1.2s to 2.0s apart, traveling at 7 m/s, using the Krauss driver model

2. Medium traffic: Cars 1.8s to 2.6s apart

3. Low traffic: Cars 2.4s to 3.2s apart

4. Moderate traffic: Cars traveling at 11 m/s

5. Fast traffic: Cars traveling at 15 m/s

(Distributions 2-5 only list the difference between them and the default traffic)

### Experiment Configurations

The results here contain all of the experiments run using the configurations in the configs folder. Generally, 3 trials were run with each traffic configuration, testing the DDPG, ST solver, and combination of DDPG + ST.

### Columns

* crashed: Crash rate
* merged: Merge rate
* mean_abs_jerk_merged: Mean absolute jerk, averaged over trials that successfully merged
* time_to_merge: Average time taken to merge successfully
* TASK: Evaluating either the DDPG, the 'ST' solver, or the combination
* TEST_ST_STRICTLY_BETTER: Whether we are applying the comparison of ST solver and RL trajectories in the combination (see section IV.b of the paper, last paragraph)
* LOG_DIR: Essentially, the name of the experiment. The numbers 1-3 denote which trial this row corresponds to
* The columns named `*_std` denote the standard error of the previous column

In [79]:
# Code to read the raw data from a csv file

import pandas as pd
import numpy as np

%matplotlib notebook
%matplotlib notebook

pd.set_option('display.width', 120)

data = pd.read_csv('saved_data.csv')

def filter_minimal_columns(data):
    relevant_column_list = [
        'crashed', 'merged', 'merged_std', 'mean_abs_jerk_merged', 'mean_abs_jerk_merged_std', 'time_to_merge', 'time_to_merge_std', 'TASK', 'NUM_EPISODES', 'TEST_ST_STRICTLY_BETTER', 'LOG_DIR'
    ]
    return data.loc[:, relevant_column_list]

# Select only the runs relevant to this experiment
def get_filtered_data(data, traffic_description, trial_name):
    filtered_data = data
    if traffic_description != 'any':
        filtered_data = data[data['TRAFFIC_DESCRIPTION']==traffic_description]
    if trial_name == 'any':
        filter_name = np.logical_not(filtered_data['MODEL_NAME'].isnull())
    else:
        filter_name = filtered_data['MODEL_NAME'].str.startswith(trial_name)
    filter_strictly_better = filtered_data['TEST_ST_STRICTLY_BETTER']
    filter_st = filtered_data['TASK'] == 'ST'
    filter_ddpg = filtered_data['TASK'] == 'EVALUATE_DDPG'
    filter_combination = filtered_data['TASK'] == 'EVALUATE_COMBINED_DDPG'
    filter_combination = np.logical_and.reduce([filter_combination, filter_name])
    filter_ddpg = np.logical_and(filter_name, filter_ddpg)
    filtered_data = filtered_data[np.logical_or.reduce([filter_st, filter_combination, filter_ddpg])]
    filtered_data = filtered_data.sort_values(by='LOG_DIR')
    return filter_minimal_columns(filtered_data)

simple_low = get_filtered_data(data, 'uniform-7.0-2.4-varying', 'runs/ddpg_low')
simple_moderate = get_filtered_data(data, 'uniform-11.0-1.2-varying', 'runs/ddpg_moderate')
simple_fast = get_filtered_data(data, 'uniform-15.0-1.2-varying', 'runs/ddpg_fast')
simple_medium = get_filtered_data(data, 'uniform-7.0-1.8-varying', 'runs/ddpg_medium')
simple_default = get_filtered_data(data, 'uniform-7.0-1.2-varying', 'runs/ddpg_default')

In [75]:
# Experiments varying the time between highway cars

print("##### Data for the low traffic #####\n")
print(simple_low)
print("\n##### Data for the medium traffic #####\n")
print(simple_medium)
print("\n##### Data for the default (heavy) traffic #####\n")
print(simple_default)

##### Data for the low traffic #####

    crashed   merged  merged_std  mean_abs_jerk_merged  mean_abs_jerk_merged_std  time_to_merge  time_to_merge_std  \
12  0.00000  1.00000    0.000000              0.653910                  0.001900      23.532450           0.034645   
77  0.00000  1.00000    0.000000              0.679092                  0.001924      23.375400           0.032983   
19  0.00000  1.00000    0.000000              0.534501                  0.002243      23.697150           0.031639   
30  0.00000  1.00000    0.000000              0.568452                  0.002626      23.584450           0.031339   
17  0.00000  1.00000    0.000000              0.419440                  0.002132      24.618100           0.028753   
80  0.00000  1.00000    0.000000              0.423902                  0.002220      24.518900           0.029859   
35  0.00325  0.99675    0.000900              0.288105                  0.000979      22.081716           0.027010   
0   0.01810  0.981

In [76]:
# Experiments varying the speed of highway cars

print("##### Data for the fast traffic #####\n")
print(filter_minimal_columns(simple_fast))
print("\n##### Data for the moderate traffic #####\n")
print(filter_minimal_columns(simple_moderate))
print("\n##### Data for the default (slow) traffic #####\n")
print(filter_minimal_columns(simple_default))

##### Data for the fast traffic #####

    crashed   merged  merged_std  mean_abs_jerk_merged  mean_abs_jerk_merged_std  time_to_merge  time_to_merge_std  \
42  0.00025  0.99975    0.000250              0.848362                  0.004201      14.705426           0.022881   
58  0.00000  1.00000    0.000000              0.799222                  0.003598      14.642700           0.022977   
43  0.00000  1.00000    0.000000              0.744983                  0.003538      14.634050           0.039720   
57  0.00000  1.00000    0.000000              0.736901                  0.003391      14.428750           0.027610   
47  0.00000  1.00000    0.000000              0.784758                  0.003069      14.127950           0.018814   
62  0.00000  1.00000    0.000000              0.795738                  0.003094      14.112900           0.018856   
36  0.00025  0.99975    0.000250              0.417873                  0.002195      13.731683           0.016022   
37  0.00225  0.99

In [77]:
# Cross-traffic experiments, trained on the moderate traffic but evaluated on different traffic speeds

print("##### Fast traffic with moderate traffic network #####\n")
traffic = get_filtered_data(data, 'uniform-15.0-1.2-varying', 'runs/ddpg_moderate')
print(traffic)

print("\n##### Slow traffic with moderate traffic network #####\n")
traffic = get_filtered_data(data, 'uniform-7.0-1.2-varying', 'runs/ddpg_moderate')
print(traffic)

##### Fast traffic with moderate traffic network #####

    crashed   merged  merged_std  mean_abs_jerk_merged  mean_abs_jerk_merged_std  time_to_merge  time_to_merge_std  \
53  0.00000  0.98800    0.001722              0.774107                  0.002827      15.619281           0.023415   
64  0.00000  1.00000    0.000000              0.803689                  0.002726      15.371200           0.022610   
51  0.00000  1.00000    0.000000              0.836248                  0.002753      14.665350           0.019811   
63  0.00000  1.00000    0.000000              0.842473                  0.002463      14.630850           0.019721   
52  0.00000  1.00000    0.000000              0.653557                  0.002270      15.500950           0.024547   
65  0.00000  1.00000    0.000000              0.651143                  0.002217      15.478250           0.024358   
67  0.68325  0.30325    0.007269              0.772585                  0.008056      14.990932           0.036819   


In [78]:
# Cross-traffic experiments, trained on the medium traffic but evaluated on different time gaps between cars

print("##### Heavy traffic with medium traffic network #####\n")
traffic = get_filtered_data(data, 'uniform-7.0-1.2-varying', 'runs/ddpg_medium')
print(traffic)

print("\n##### Low traffic with medium traffic network #####\n")
traffic = get_filtered_data(data, 'uniform-7.0-2.4-varying', 'runs/ddpg_medium')
print(traffic)


##### Heavy traffic with medium traffic network #####

    crashed   merged  merged_std  mean_abs_jerk_merged  mean_abs_jerk_merged_std  time_to_merge  time_to_merge_std  \
76  0.00000  1.00000    0.000000              0.993027                  0.002914      27.648550           0.034160   
75  0.00175  0.99825    0.000661              1.127848                  0.002026      24.702780           0.024548   
78  0.00000  1.00000    0.000000              0.683306                  0.002934      27.209100           0.044511   
84  0.07475  0.92525    0.004159              0.366821                  0.001092      24.195677           0.032083   
83  0.21200  0.78800    0.006463              0.479483                  0.002360      23.291624           0.027018   
33  0.08300  0.91700    0.004363              0.547102                  0.002032      23.149073           0.029679   
18  0.00000  1.00000    0.000000              1.105359                  0.001918      29.837650           0.014834   

